<a href="https://colab.research.google.com/github/KIMNAMHYEON-Kpass/love-letter-lab/blob/main/notebooks/01_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개요

## 프롬프트

- '# 개요'의 상단까지가 기존 수업에서 진행한 워크플로우에 해당합니다.
- 해당 워크플로우에 맞춰서 이번 작업도 진행해주세요.
- 다음 조건대로 Hugging Face GPT-류 디코더 모델을 사용해 연애 편지를 생성하는 Colab 코드 셀을 만들어주세요.

[사전 조건]
- 아래와 같이 Colab 코드에서 prompt 변수를 만들고, 제시된 내용이 들어가게 해주세요.
- name, relation, situation, tone, length 값은 변수로 설정하여 바꿀 수 있도록 하고, f-string 형식으로 prompt에 삽입되도록 해주세요.

[프롬프트 내용]

"""

당신은 연애 편지 작가입니다.

[제약 조건]
- 반드시 존댓말로 작성합니다.
- 과장된 비유나 지나친 감정 표현을 피합니다.
- 톤은 {tone}로 유지합니다.
- 총 {length} 문장으로 작성합니다.
- 느낌표는 최대 1번만 사용합니다.
- 이모지는 사용하지 않습니다.

[상황 정보]
- 받는 사람 이름: {name}
- 관계: {relation}
- 상황: {situation}
위 정보를 모두 반영하여 자연스럽고 따뜻한 연애 편지를 작성하세요.
편지는 자연스러운 문단으로 출력하고, 인사말이나 서명은 넣지 마세요.

"""

[요구사항]
1. Hugging Face Transformers 라이브러리 로드
2. 적절한 한국어 지원 GPT-류 모델 불러오기 (예: skt/kogpt2-base-v2 또는 소형 LLM)
3. max_new_tokens, temperature, top_p 등 하이퍼파라미터를 조절해 생성
4. 생성 결과를 깔끔하게 print하도록 구성

- 추가적으로 Colab 리소스 절약을 위해서, CPU 런타임을 통해 진행하다가, 최후의 학습(필요 시)과 실제 결과물 생성에만 "T4" 런타임으로 변경하여 모두 실행하는 형식으로 진행할 예정이니 그것을 감안해서 작성해주세요.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/인공지능사관학교/2508 AI익스퍼트'

/content/drive/MyDrive/인공지능사관학교/2508 AI익스퍼트


In [3]:
# 편지 생성 프롬프트 예시
name = "민수"
relation = "동기"
situation = "발표를 앞두고 긴장한 상태"
tone = "달달"
length = 3  # 문장 수

prompt = f"""
당신은 연애 편지 작가입니다.
[제약 조건]
- 반드시 존댓말로 작성합니다.
- 과장된 비유나 지나친 감정 표현을 피합니다.
- 톤은 {tone}로 유지합니다.
- 총 {length} 문장으로 작성합니다.
- 느낌표는 최대 1번만 사용합니다.
- 이모지는 사용하지 않습니다.
[상황 정보]
- 받는 사람 이름: {name}
- 관계: {relation}
- 상황: {situation}
위 정보를 모두 반영하여 자연스럽고 따뜻한 연애 편지를 작성하세요.
편지는 자연스러운 문단으로 출력하고, 인사말이나 서명은 넣지 마세요.
"""
print(prompt)


당신은 연애 편지 작가입니다.
[제약 조건]
- 반드시 존댓말로 작성합니다.
- 과장된 비유나 지나친 감정 표현을 피합니다.
- 톤은 달달로 유지합니다.
- 총 3 문장으로 작성합니다.
- 느낌표는 최대 1번만 사용합니다.
- 이모지는 사용하지 않습니다.
[상황 정보]
- 받는 사람 이름: 민수
- 관계: 동기
- 상황: 발표를 앞두고 긴장한 상태
위 정보를 모두 반영하여 자연스럽고 따뜻한 연애 편지를 작성하세요.
편지는 자연스러운 문단으로 출력하고, 인사말이나 서명은 넣지 마세요.



In [4]:
!pip install transformers accelerate sentencepiece

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "skt/kogpt2-base-v2"  # 또는 원하는 GPT류 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [5]:
# 3. 하이퍼파라미터를 조절해 생성
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

# max_new_tokens, temperature, top_p 등 하이퍼파라미터 조절
# length 변수를 활용하여 max_new_tokens 설정 (대략적인 문장 수 조절)
# 한국어 문장 길이를 고려하여 적절한 토큰 수를 설정해야 합니다.
# 여기서는 length * 30 정도로 설정 (조절 필요)
max_tokens = length * 30

output_sequences = model.generate(
    input_ids=input_ids,
    max_new_tokens=max_tokens,
    temperature=0.7,  # 창의성 조절 (낮을수록 보수적)
    top_p=0.9,        # 상위 p% 확률 mass를 가진 토큰만 샘플링
    num_return_sequences=1,
    no_repeat_ngram_size=2, # 반복 방지
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id # 패딩 토큰 설정 (warning 방지)
)

# 4. 생성 결과를 깔끔하게 print하도록 구성
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)

# 프롬프트 부분을 제외하고 생성된 편지 내용만 추출 (간단한 문자열 처리)
# 실제 사용 시에는 모델의 출력 특성을 고려하여 더 정교한 파싱 필요
letter_start_index = generated_text.find("편지는 자연스러운 문단으로 출력하고, 인사말이나 서명은 넣지 마세요.") + len("편지는 자연스러운 문단으로 출력하고, 인사말이나 서명은 넣지 마세요.")
generated_letter = generated_text[letter_start_index:].strip()

print(generated_letter)

이모지는 절대 사용하지 마세요. 현재 스맥다운의 메인 스토리는 2016년 7월 23일에 방영되었다.
다만 이 에피소드는 스맥이 게임 개발팀을 만들어서, 게임의 개발 과정에 관여를 하는 것으로 보인다.
스맥다운이 게임을 개발할 때 감독은 스맥을 개발중이라고 말하는 반면, 스맥은 게임 개발을 하고 있지 않다고 말하는 등 스멕다운은 게임개발에 관여하지 않는다고 한다.
게임 개발자인 스킴의 말에
